In [33]:
import os
from os import listdir
from os.path import isfile, join
import glob
import string
import operator
import csv
import pandas as pd
import calendar
import numpy as np

In [2]:
homedir = os.path.expanduser("~")
path_to_data = homedir+"\\Assignment 3\\Data"
print(path_to_data)

C:\Users\priya\Assignment 3\Data


In [17]:
output_dist={}
for subdir,dirs, files in os.walk(path_to_data):
    #print(files)
    for file in files:
        if file == 'cricket_matches.csv':
            #print(file)
            csv_data1=pd.read_csv(os.path.join(subdir,file))

,match_details,result,scores,date,venue,round,home,away,winner,win_by_runs,...,innings1_wickets,innings1_overs_batted,innings1_overs,innings2,innings2_runs,innings2_wickets,innings2_overs_batted,innings2_overs,D/L_method,target
0,North Group:Durham v Nottinghamshire at Cheste...,Durham won by 6 wickets (with 5 balls remaining),Nottinghamshire 157/7 (20/20 ov); Durham 160/4...,13/06/2003,Chester-le-Street,North Group,Durham,Nottinghamshire,Durham,NaN,...,7.0,20.0,20.0,Durham,160.0,4.0,19.1,20.0,NaN,NaN
1,South Group:Hampshire v Sussex at Southampton-...,Hampshire won by 5 runs,Hampshire 153 (19.4/20 ov); Sussex 148/7 (20/2...,13/06/2003,Southampton,South Group,Hampshire,Sussex,Hampshire,5.0,...,10.0,19.4,20.0,Sussex,148.0,7.0,20.0,20.0,NaN,NaN


## Making new frame with relevant columns 

In [18]:
match_details_df = csv_data1[["home","away","winner","innings1","innings1_runs","innings2","innings2_runs"]].copy()
print(match_details_df.head(5))

,home,away,winner,innings1,innings1_runs,innings2,innings2_runs
0,Durham,Nottinghamshire,Durham,Nottinghamshire,157.0,Durham,160.0
1,Hampshire,Sussex,Hampshire,Hampshire,153.0,Sussex,148.0
2,Somerset,Warwickshire,Warwickshire,Warwickshire,188.0,Somerset,169.0
3,Surrey,Middlesex,Surrey,Middlesex,155.0,Surrey,158.0
4,Worcestershire,Northamptonshire,Worcestershire,Northamptonshire,150.0,Worcestershire,151.0


## Getting only those records which are played at home and won the match

In [19]:
home_winners_df = match_details_df[(match_details_df['home'] == match_details_df['winner'])]

,home,away,winner,innings1,innings1_runs,innings2,innings2_runs
0,Durham,Nottinghamshire,Durham,Nottinghamshire,157.0,Durham,160.0
1,Hampshire,Sussex,Hampshire,Hampshire,153.0,Sussex,148.0
3,Surrey,Middlesex,Surrey,Middlesex,155.0,Surrey,158.0
4,Worcestershire,Northamptonshire,Worcestershire,Northamptonshire,150.0,Worcestershire,151.0
5,Gloucestershire,Worcestershire,Gloucestershire,Worcestershire,122.0,Gloucestershire,126.0


### Getting the avg. if they have batted in first innings.  

In [20]:
home_inning1 = home_winners_df[["home","innings1","innings1_runs"]].copy()
home_inning1 = home_inning1[(home_inning1['home'] == home_inning1['innings1'])]

In [21]:
home_inning1 = home_inning1.groupby('home', as_index=False)['innings1_runs'].mean()
print(home_inning1.head(5))

,home,innings1_runs
0,Abahani Limited,172.000000
1,Abbottabad Falcons,155.333333
2,Abbottabad Region,198.000000
3,Abbottabad Rhinos,186.000000
4,Adelaide Strikers,183.333333


### getting the aveg. If they batted in second innings 

In [22]:
home_inning2 = home_winners_df[["home","innings2","innings2_runs"]].copy()
home_inning2 = home_inning2[(home_inning2['home'] == home_inning2['innings2'])]

In [23]:
home_inning2 = home_inning2.groupby('home', as_index=False)['innings2_runs'].mean()
print(home_inning2.head(5))

,home,innings2_runs
0,Abbottabad Falcons,120.400000
1,Abbottabad Region,175.000000
2,Abbottabad Rhinos,144.000000
3,Adelaide Strikers,153.000000
4,Afghanistan,126.777778


- Combining both results using outer join 

In [24]:
final_inning_df = pd.merge(home_inning1,home_inning2,on ='home' , how='outer')
final_inning_df.columns = ["home","First_inning_avg","Second_inning_avg"]

,home,First_inning_avg,Second_inning_avg
0,Abahani Limited,172.000000,NaN
1,Abbottabad Falcons,155.333333,120.4
2,Abbottabad Region,198.000000,175.0
3,Abbottabad Rhinos,186.000000,144.0
4,Adelaide Strikers,183.333333,153.0


- Calculating final aveg.

In [27]:
final_inning_df["Total"] = final_inning_df[["First_inning_avg","Second_inning_avg"]].mean(axis=1)

In [35]:
final_inning_df.drop('First_inning_avg', axis=1, inplace=True)
final_inning_df.drop('Second_inning_avg', axis=1, inplace=True)
final_inning_df["Total"] = final_inning_df["Total"].apply(np.int64)
print(final_inning_df.head(5))

,home,Total
0,Abahani Limited,172
1,Abbottabad Falcons,137
2,Abbottabad Region,186
3,Abbottabad Rhinos,165
4,Adelaide Strikers,168


In [30]:
def save_to_csv():
    if not os.path.exists(homedir+"\\Assignment 3\\Output\\Question3"):
        os.makedirs(homedir+"\\Assignment 3\\Output\\Question3")
    with open(homedir+'\\Assignment 3\\Output\\Question3\\Q3_ouptput.csv', 'w') as myfile:
        myfile.write(final_inning_df.to_csv(index=False))

In [ ]:
save_to_csv()